In [ ]:
from docx import Document
import os

# Crear documento Word
doc = Document()
doc.add_heading("Informe de Proceso ETL - Jardinería", 0)

sections = [
    ("1. Preparación", [
        "1.1 Revisión del modelo estrella:",
        "El modelo estrella se compone de una tabla de hechos central llamada `detalle_pedido` y varias tablas de dimensiones: producto, cliente, empleado, oficina, pedido, pago.",
        "Este modelo facilita consultas analíticas rápidas y agregaciones eficientes.",
        "",
        "1.2 Verificación de la base de datos staging:",
        "Se comprobaron las tablas clientes_staging, productos_staging, pedidos_staging y detalles_pedidos. Todas presentan esquemas compatibles con los datos fuente."
    ]),
    ("2. Extracción de datos", [
        "Se realizaron las siguientes extracciones:",
        "- Clientes con separación de nombre y apellido.",
        "- Productos con sus precios y stocks.",
        "- Pedidos y detalles de pedidos.",
        "Las consultas utilizadas aseguran una correcta migración y preparación para la transformación."
    ]),
    ("3. Transformación", [
        "- Limpieza de datos (emails vacíos, normalización de texto, eliminación de duplicados).",
        "- Subtotales calculados dinámicamente.",
        "- Tipificación y estandarización de cadenas."
    ]),
    ("4. Carga al Data Mart", [
        "Los datos transformados se insertaron en el Data Mart mediante consultas `INSERT INTO ... SELECT`.",
        "Se realizaron verificaciones posteriores para confirmar la integridad y coherencia de los datos cargados."
    ]),
    ("5. Documentación y Presentación", [
        "- Código SQL completo utilizado para ETL.",
        "- Diagramas del modelo estrella y del esquema de staging.",
        "- Explicaciones detalladas de cada paso con justificación técnica."
    ]),
    ("Ventajas del enfoque:", [
        "- Control y trazabilidad en cada etapa.",
        "- Separación clara entre fuente, staging y destino.",
        "- Permite detección y corrección temprana de errores.",
        "- Compatible con escalabilidad futura y herramientas de automatización (SSIS, ADF)."
    ])
]

for title, points in sections:
    doc.add_heading(title, level=1)
    for p in points:
        doc.add_paragraph(p, style="BodyText")

# Guardar como .docx
doc_path = "/mnt/data/Informe_ETL_Jardineria.docx"
doc.save(doc_path)

doc_path


In [ ]:
from docx import Document
import os

# Crear documento Word
doc = Document()
doc.add_heading("Informe de Proceso ETL - Jardinería", 0)

sections = [
    ("1. Preparación", [
        "1.1 Revisión del modelo estrella:",
        "El modelo estrella se compone de una tabla de hechos central llamada `detalle_pedido` y varias tablas de dimensiones: producto, cliente, empleado, oficina, pedido, pago.",
        "Este modelo facilita consultas analíticas rápidas y agregaciones eficientes.",
        "",
        "1.2 Verificación de la base de datos staging:",
        "Se comprobaron las tablas clientes_staging, productos_staging, pedidos_staging y detalles_pedidos. Todas presentan esquemas compatibles con los datos fuente."
    ]),
    ("2. Extracción de datos", [
        "Se realizaron las siguientes extracciones:",
        "- Clientes con separación de nombre y apellido.",
        "- Productos con sus precios y stocks.",
        "- Pedidos y detalles de pedidos.",
        "Las consultas utilizadas aseguran una correcta migración y preparación para la transformación."
    ]),
    ("3. Transformación", [
        "- Limpieza de datos (emails vacíos, normalización de texto, eliminación de duplicados).",
        "- Subtotales calculados dinámicamente.",
        "- Tipificación y estandarización de cadenas."
    ]),
    ("4. Carga al Data Mart", [
        "Los datos transformados se insertaron en el Data Mart mediante consultas `INSERT INTO ... SELECT`.",
        "Se realizaron verificaciones posteriores para confirmar la integridad y coherencia de los datos cargados."
    ]),
    ("5. Documentación y Presentación", [
        "- Código SQL completo utilizado para ETL.",
        "- Diagramas del modelo estrella y del esquema de staging.",
        "- Explicaciones detalladas de cada paso con justificación técnica."
    ]),
    ("Ventajas del enfoque:", [
        "- Control y trazabilidad en cada etapa.",
        "- Separación clara entre fuente, staging y destino.",
        "- Permite detección y corrección temprana de errores.",
        "- Compatible con escalabilidad futura y herramientas de automatización (SSIS, ADF)."
    ])
]

for title, points in sections:
    doc.add_heading(title, level=1)
    for p in points:
        doc.add_paragraph(p, style="BodyText")

# Guardar como .docx
doc_path = "E:/Tatan/Iudigital/Ingenieria/Base de Datos II/Cuarta Actividad/Informe_ETL_Jardineria.docx"
doc.save(doc_path)

doc_path


In [ ]:
# Crear los scripts SQL para extracción, transformación y carga
extract_sql = """
-- EXTRACCIÓN DE DATOS HACIA STAGING

-- Clientes
INSERT INTO staging.dbo.clientes_staging (id_cliente, nombre, apellido, telefono, email, ciudad)
SELECT id_cliente, 
       PARSENAME(REPLACE(nombre_contacto, ' ', '.'), 2) AS nombre, 
       PARSENAME(REPLACE(nombre_contacto, ' ', '.'), 1) AS apellido, 
       telefono, 
       email, 
       ciudad
FROM jardineria.dbo.cliente;

-- Productos
INSERT INTO staging.dbo.productos_staging (id_producto, nombre_producto, descripcion, precio, stock)
SELECT id_producto, nombre, descripcion, precio_venta, cantidad_en_stock
FROM jardineria.dbo.producto;

-- Pedidos
INSERT INTO staging.dbo.pedidos_staging (id_pedido, id_cliente, fecha_pedido, total)
SELECT id_pedido, codigo_cliente, fecha_pedido, NULL
FROM jardineria.dbo.pedido;

-- Detalles de pedidos
INSERT INTO staging.dbo.detalles_pedidos (id_pedido, id_producto, cantidad, subtotal)
SELECT id_pedido, codigo_producto, cantidad, cantidad * precio_unidad
FROM jardineria.dbo.detalle_pedido;
"""

transform_sql = """
-- TRANSFORMACIÓN DE DATOS EN STAGING

-- Corrección de emails vacíos
UPDATE staging.dbo.clientes_staging
SET email = NULL
WHERE email = '';

-- Estandarización de nombres y apellidos
UPDATE staging.dbo.clientes_staging
SET nombre = UPPER(LTRIM(RTRIM(nombre))),
    apellido = UPPER(LTRIM(RTRIM(apellido)));
"""

load_sql = """
-- CARGA AL DATA MART FINAL

-- Clientes
INSERT INTO mart.dbo.cliente (id_cliente, nombre, apellido, telefono, email, ciudad)
SELECT * FROM staging.dbo.clientes_staging;

-- Productos
INSERT INTO mart.dbo.producto (id_producto, nombre, descripcion, precio, stock)
SELECT * FROM staging.dbo.productos_staging;

-- Pedidos
INSERT INTO mart.dbo.pedido (id_pedido, id_cliente, fecha_pedido, total)
SELECT * FROM staging.dbo.pedidos_staging;

-- Detalles de pedidos
INSERT INTO mart.dbo.detalle_pedido (id_pedido, id_producto, cantidad, subtotal)
SELECT * FROM staging.dbo.detalles_pedidos;
"""

# Guardar los scripts en archivos .sql
extract_path = "E:\Tatan\Iudigital\Ingenieria\Base de Datos II\Cuarta Actividad\script.sql"
transform_path = "E:\Tatan\Iudigital\Ingenieria\Base de Datos II\Cuarta Actividad\02_transformacion.sql"
load_path = "E:\Tatan\Iudigital\Ingenieria\Base de Datos II\Cuarta Actividad\exportes "

with open(extract_path, "w") as f:
    f.write(extract_sql)

with open(transform_path, "w") as f:
    f.write(transform_sql)

with open(load_path, "w") as f:
    f.write(load_sql)

extract_path, transform_path, load_path
